<img src="barbiepng2.png" width="200" height="200"/> <font size="10" color='E0218A'>search engine</font>
<br>
<br>
<font size="4" color='E0218A'>Κατσιώνη Άννα    -   21390083</font>
<br>
<font size="4" color='E0218A'>Καζάλα Σταυρούλα - 21390071</font>
<br>
<br>

<font size="6" color='E0218A'><ins>Βήμα 1. Συλλογή δεδομένων:</ins></font>

Αρχικά, με την χρήση BeautifulSoup μπαίνουμε στο άρθρο της της wikepedia και αποθηκεύουμε όλα τα links της πρώτης στήλης του σχετικού πίνακα στο αρχείο barbie_movies.csv .

In [13]:
import requests
from bs4 import BeautifulSoup
import csv

#The url we will scrape links from
url = 'https://en.wikipedia.org/wiki/Barbie_(media_franchise)'


# Send a GET request to fetch the HTML content
response = requests.get(url).text
soup = BeautifulSoup(response, 'lxml')

# Find the first table on the page (adjust as needed)
table = soup.find('table', 'wikitable')

links_list = []

for row in table.find_all('tr')[1:]:  # Skip the header row (if present)
    cells = row.find_all('td')
    # Find all links in the second column
    links = cells[1].find_all('a', href=True)
    for link in links:
       if('wiki' in link['href']): #throw out invalid links
        full_url = "https://en.wikipedia.org" + link['href']
        links_list.append(full_url)

#print(links_list)

with open('barbie_movies.csv', mode='w', newline='', encoding='utf-8') as file:
   writer = csv.writer(file)
   writer.writerow(["Title", "Summary","URL"])

   title_list=[]

   for link in links_list:
      
      page_soup = BeautifulSoup(requests.get(link).text, 'html.parser')
      title = page_soup.find(id='firstHeading')
      if title not in title_list: #Some arcticles appear multiple times. We only want them once.
         title_list.append(title)
         summary = ""
         num_pars=0
         paragraphs = page_soup.find_all('p')
         for paragraph in paragraphs:
            summary =  summary+ " " + (paragraph.text).strip()
            num_pars=num_pars+1
            if num_pars > 2:  # Collect the first two paragraphs
               break
         writer.writerow([title.string, summary, link])

print("Summaries have been written to barbie_movies.csv")

Summaries have been written to barbie_movies.csv


Αποθηκεύουμε τα περιεχόμενα του csv αρχείου στη δομή δεδομένων data και προβάλλουμε τα περιεχόμενά του:

In [14]:

import pandas as pd
import nltk
nltk.download('stopwords') 

data = pd.read_csv("barbie_movies.csv")
data.head()



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Stav\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Title,Summary,URL
0,Barbie in the Nutcracker,Barbie in the Nutcracker is a 2001 animated ...,https://en.wikipedia.org/wiki/Barbie_in_the_Nu...
1,Barbie as Rapunzel,Barbie as Rapunzel is a 2002 animated fairy ...,https://en.wikipedia.org/wiki/Barbie_as_Rapunzel
2,Barbie of Swan Lake,Barbie of Swan Lake is a 2003 animated fantas...,https://en.wikipedia.org/wiki/Barbie_of_Swan_Lake
3,Barbie as the Princess and the Pauper,Barbie as the Princess and the Pauper is a 2...,https://en.wikipedia.org/wiki/Barbie_as_the_Pr...
4,Barbie: Fairytopia,Barbie: Fairytopia is a 2005 animated fantas...,https://en.wikipedia.org/wiki/Barbie:_Fairytopia


<font size="6" color='E0218A'><ins>Βήμα 2. Προεπεξεργασία κειμένου (Text Processing):</ins></font>

Με την εντολή nltk.word_tokenize, ένα κείμενο χωρίζεται σε tokens.

Για παράδειγμα, παρακάτω κάνουμε tokenize το στοιχείο 2 της στήλης "Summary" (περίληψη), του csv:

In [32]:
import nltk

nltk.word_tokenize(data["Summary"][2])

['Barbie',
 'of',
 'Swan',
 'Lake',
 'is',
 'a',
 '2003',
 'animated',
 'fantasy',
 'film',
 'co-produced',
 'by',
 'Mainframe',
 'Entertainment',
 'and',
 'Mattel',
 'Entertainment',
 ',',
 'and',
 'distributed',
 'by',
 'Artisan',
 'Home',
 'Entertainment',
 '.',
 'Based',
 'on',
 'the',
 'Tchaikovsky',
 'ballet',
 'Swan',
 'Lake',
 ',',
 '[',
 '2',
 ']',
 '[',
 '3',
 ']',
 'it',
 'is',
 'the',
 'third',
 'in',
 'the',
 'Barbie',
 'film',
 'series',
 ',',
 'with',
 'Kelly',
 'Sheridan',
 'providing',
 'the',
 'voice',
 'of',
 'Barbie',
 '.',
 '[',
 '4',
 ']',
 'Barbie',
 'of',
 'Swan',
 'Lake',
 'was',
 'released',
 'on',
 'VHS',
 'and',
 'DVD',
 'on',
 'September',
 '30',
 ',',
 '2003',
 'by',
 'Artisan',
 ',',
 'later',
 'making',
 'a',
 'television',
 'premiere',
 'on',
 'Nickelodeon',
 'on',
 'November',
 '16',
 ',',
 '2003',
 '.',
 '[',
 '5',
 ']',
 'and',
 'was',
 'subsequently',
 'released',
 'overseas',
 'through',
 'Entertainment',
 'Rights',
 'and',
 'Universal',
 'Pictures

Επαναλαμβάνουμε τη διαδικασία για κάθε γραμμή της στήλης Summary, και αποθηκεύουμε όλα τα tokens στη λίστα "tokenized":

In [16]:

tokenized=[]
for i in range(0,len(data)) :
    tokenized=tokenized+nltk.word_tokenize(data["Summary"][i])

print(tokenized)

    

['Barbie', 'in', 'the', 'Nutcracker', 'is', 'a', '2001', 'animated', 'fantasy', 'film', 'co-produced', 'by', 'Mainframe', 'Entertainment', 'and', 'Mattel', 'Entertainment', ',', 'and', 'distributed', 'by', 'Artisan', 'Home', 'Entertainment', '.', 'Loosely', 'adapted', 'from', 'E.', 'T.', 'A.', 'Hoffmann', "'s", '1816', 'short', 'story', ',', '``', 'The', 'Nutcracker', 'and', 'the', 'Mouse', 'King', "''", ',', '[', '4', ']', 'and', 'featuring', 'music', 'based', 'from', 'Tchaikovsky', "'s", '1892', 'ballet', ',', 'The', 'Nutcracker', ',', 'it', 'marked', 'the', 'character', "'s", 'first', 'film', 'since', 'the', '1987', 'television', 'specials', ';', 'Barbie', 'and', 'the', 'Rockers', ':', 'Out', 'of', 'This', 'World', 'and', 'Barbie', 'and', 'the', 'Sensations', ':', 'Rockin', "'", 'Back', 'to', 'Earth', ',', 'with', 'Kelly', 'Sheridan', 'providing', 'the', 'voice', 'of', 'Barbie', '.', '[', '5', ']', 'It', 'was', 'also', 'the', 'first', 'Barbie', 'movie', 'to', 'be', 'computer-animate

Έπειτα, μετατρέπουμε όλες τις λέξεις της λίστας tokenized σε πεζά γράμματα:

In [17]:
lower_words=[]

lower_words = [x.lower() for x in tokenized]
print(lower_words)

['barbie', 'in', 'the', 'nutcracker', 'is', 'a', '2001', 'animated', 'fantasy', 'film', 'co-produced', 'by', 'mainframe', 'entertainment', 'and', 'mattel', 'entertainment', ',', 'and', 'distributed', 'by', 'artisan', 'home', 'entertainment', '.', 'loosely', 'adapted', 'from', 'e.', 't.', 'a.', 'hoffmann', "'s", '1816', 'short', 'story', ',', '``', 'the', 'nutcracker', 'and', 'the', 'mouse', 'king', "''", ',', '[', '4', ']', 'and', 'featuring', 'music', 'based', 'from', 'tchaikovsky', "'s", '1892', 'ballet', ',', 'the', 'nutcracker', ',', 'it', 'marked', 'the', 'character', "'s", 'first', 'film', 'since', 'the', '1987', 'television', 'specials', ';', 'barbie', 'and', 'the', 'rockers', ':', 'out', 'of', 'this', 'world', 'and', 'barbie', 'and', 'the', 'sensations', ':', 'rockin', "'", 'back', 'to', 'earth', ',', 'with', 'kelly', 'sheridan', 'providing', 'the', 'voice', 'of', 'barbie', '.', '[', '5', ']', 'it', 'was', 'also', 'the', 'first', 'barbie', 'movie', 'to', 'be', 'computer-animate

Αφαιρούμε τα stopwords (συνήθεις λέξεις όπωςς and, the, I κ.λ.π.) από τη λίστα lower_words, δημιουργόντας μία νέα λίστα cleaned_words:

In [18]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 
cleaned_words=[]

for words in lower_words:
    if words not in stop_words:
        cleaned_words.append(words)
print(cleaned_words)

['barbie', 'nutcracker', '2001', 'animated', 'fantasy', 'film', 'co-produced', 'mainframe', 'entertainment', 'mattel', 'entertainment', ',', 'distributed', 'artisan', 'home', 'entertainment', '.', 'loosely', 'adapted', 'e.', 't.', 'a.', 'hoffmann', "'s", '1816', 'short', 'story', ',', '``', 'nutcracker', 'mouse', 'king', "''", ',', '[', '4', ']', 'featuring', 'music', 'based', 'tchaikovsky', "'s", '1892', 'ballet', ',', 'nutcracker', ',', 'marked', 'character', "'s", 'first', 'film', 'since', '1987', 'television', 'specials', ';', 'barbie', 'rockers', ':', 'world', 'barbie', 'sensations', ':', 'rockin', "'", 'back', 'earth', ',', 'kelly', 'sheridan', 'providing', 'voice', 'barbie', '.', '[', '5', ']', 'also', 'first', 'barbie', 'movie', 'computer-animated', '.', 'barbie', 'rapunzel', '2002', 'animated', 'fairy', 'tale', 'film', 'co-produced', 'mainframe', 'entertainment', 'mattel', 'entertainment', ',', 'distributed', 'artisan', 'home', 'entertainment', '.', 'adaptation', '1812', 'germ

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Stav\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Το κείμενο είναι scraped από τη Wikipedia, οπότε περιέχει αναφορές με τη μορφή αριθμών εντός αγκύλης.

Επειδή δεν επιθυμούμε να αφαιρέσουμε όλους τους αριθμούς από τα tokens (σε περίπτωση που ο χρήστης θέλει να αναζητήσει ταινίες με βάση τη χρονολογία), εντοπίζουμε τους αριθμούς που βρίσκονται μέσα σε αγκύλες και τους αφαιρούμε.
Αυτή η διαδικασία πρέπει προφανώς να γίνει πριν την αφαίρεση των σημείων στίξης.

In [19]:
no_stray_digits = []

for i in range(len(cleaned_words)):
    if not cleaned_words[i].isdigit():
        no_stray_digits.append(cleaned_words[i])
    else:
        if not (cleaned_words[i-1] == "["):
           no_stray_digits.append(cleaned_words[i]) 
print(no_stray_digits)

['barbie', 'nutcracker', '2001', 'animated', 'fantasy', 'film', 'co-produced', 'mainframe', 'entertainment', 'mattel', 'entertainment', ',', 'distributed', 'artisan', 'home', 'entertainment', '.', 'loosely', 'adapted', 'e.', 't.', 'a.', 'hoffmann', "'s", '1816', 'short', 'story', ',', '``', 'nutcracker', 'mouse', 'king', "''", ',', '[', ']', 'featuring', 'music', 'based', 'tchaikovsky', "'s", '1892', 'ballet', ',', 'nutcracker', ',', 'marked', 'character', "'s", 'first', 'film', 'since', '1987', 'television', 'specials', ';', 'barbie', 'rockers', ':', 'world', 'barbie', 'sensations', ':', 'rockin', "'", 'back', 'earth', ',', 'kelly', 'sheridan', 'providing', 'voice', 'barbie', '.', '[', ']', 'also', 'first', 'barbie', 'movie', 'computer-animated', '.', 'barbie', 'rapunzel', '2002', 'animated', 'fairy', 'tale', 'film', 'co-produced', 'mainframe', 'entertainment', 'mattel', 'entertainment', ',', 'distributed', 'artisan', 'home', 'entertainment', '.', 'adaptation', '1812', 'german', 'fair

Αφαιρούμε όλους τους ειδικούς χαρακτήρες:

In [20]:
char_list = [';', ':', '!', "*", "-", "_", "+", '.', ',', "<", ">", "'", "(", ")", "&", "$", "[", "]", "`", "~", "’", "–" ]
cleaner_words=[]
for word in no_stray_digits:
    check=0
    for token in char_list:
        if token in word:
            check=check+1
    if check==0:
        cleaner_words.append(word)
print(cleaner_words)

['barbie', 'nutcracker', '2001', 'animated', 'fantasy', 'film', 'mainframe', 'entertainment', 'mattel', 'entertainment', 'distributed', 'artisan', 'home', 'entertainment', 'loosely', 'adapted', 'hoffmann', '1816', 'short', 'story', 'nutcracker', 'mouse', 'king', 'featuring', 'music', 'based', 'tchaikovsky', '1892', 'ballet', 'nutcracker', 'marked', 'character', 'first', 'film', 'since', '1987', 'television', 'specials', 'barbie', 'rockers', 'world', 'barbie', 'sensations', 'rockin', 'back', 'earth', 'kelly', 'sheridan', 'providing', 'voice', 'barbie', 'also', 'first', 'barbie', 'movie', 'barbie', 'rapunzel', '2002', 'animated', 'fairy', 'tale', 'film', 'mainframe', 'entertainment', 'mattel', 'entertainment', 'distributed', 'artisan', 'home', 'entertainment', 'adaptation', '1812', 'german', 'fairy', 'tale', 'rapunzel', 'brothers', 'grimm', 'second', 'barbie', 'film', 'series', 'kelly', 'sheridan', 'providing', 'voice', 'barbie', 'barbie', 'swan', 'lake', '2003', 'animated', 'fantasy', '

Παρακάτω παρουσιάζεται το αποτέλεσμα του stemming στην αρχική λίστα των tokenized λέξεων. Επιλέξαμε να μην εφαρμόσουμε αυτήν την τεχνική τελικά, καθώς "έκοβε" πολλές σημαντικές λέξεις.

In [21]:
porter = nltk.PorterStemmer()
print([porter.stem(t) for t in tokenized])

['barbi', 'in', 'the', 'nutcrack', 'is', 'a', '2001', 'anim', 'fantasi', 'film', 'co-produc', 'by', 'mainfram', 'entertain', 'and', 'mattel', 'entertain', ',', 'and', 'distribut', 'by', 'artisan', 'home', 'entertain', '.', 'loos', 'adapt', 'from', 'e.', 't.', 'a.', 'hoffmann', "'s", '1816', 'short', 'stori', ',', '``', 'the', 'nutcrack', 'and', 'the', 'mous', 'king', "''", ',', '[', '4', ']', 'and', 'featur', 'music', 'base', 'from', 'tchaikovski', "'s", '1892', 'ballet', ',', 'the', 'nutcrack', ',', 'it', 'mark', 'the', 'charact', "'s", 'first', 'film', 'sinc', 'the', '1987', 'televis', 'special', ';', 'barbi', 'and', 'the', 'rocker', ':', 'out', 'of', 'thi', 'world', 'and', 'barbi', 'and', 'the', 'sensat', ':', 'rockin', "'", 'back', 'to', 'earth', ',', 'with', 'kelli', 'sheridan', 'provid', 'the', 'voic', 'of', 'barbi', '.', '[', '5', ']', 'it', 'wa', 'also', 'the', 'first', 'barbi', 'movi', 'to', 'be', 'computer-anim', '.', 'barbi', 'as', 'rapunzel', 'is', 'a', '2002', 'anim', 'fai

<font size="6" color='E0218A'><ins>Βήμα 3. Ευρετήριο (Indexing):</ins></font>

Δημιουργούμε ένα ανεστραμμένο ευρετήριο με τα tokens της λίστας cleaner_words και το αποθηκεύουμε σε λεξικό (δομή δεδομένων dictionary, με όνομα dictionary):

In [22]:
dictionary = {}
check=0 
for i in range(len(data["Summary"])):
    token = data["Summary"][i].lower()
   
    for word in cleaner_words:
        
        if word in token:
            if word not in dictionary and word in cleaner_words:
                dictionary[word] = []   
                #print(word)
            if word in dictionary:
                if i+1 not in dictionary[word]:
                    dictionary[word].append(i+1)
                    
                
for key, value in dictionary.items():
    print(f"{key}: {value}")


barbie: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
nutcracker: [1]
2001: [1]
animated: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
fantasy: [1, 3, 4, 5, 12, 13, 16, 17, 18, 19, 20, 24]
film: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
mainframe: [1, 2, 3, 6, 7, 9, 22, 23]
entertainment: [1, 2, 3, 5, 6, 7, 8, 10, 11, 12, 14, 15, 16, 17, 18, 19, 21, 23]
mattel: [1, 2, 3, 6, 9, 10, 12, 14, 15, 16, 17, 18, 21, 23, 25, 26, 27]
distributed: [1, 2, 3, 5, 7, 8, 12, 22]
artisan: [1, 2, 3]
home: [1, 2, 3, 7, 8, 19, 21, 28]
loosely: [1, 4, 8]
adapted: [1]
hoffmann: [1]
1816: [1]
short: [1]
story: [1, 5, 6, 10, 11, 12, 17, 18, 20, 21, 24]
mouse: [1]
king: [1, 3, 6, 7, 12, 14, 16, 17, 18, 19, 20, 21, 24, 25]
featuring: [1, 19]
music: [1, 4, 8, 9, 10, 19, 20, 23, 24, 25, 26]
based: [1, 3, 5, 6, 8, 9, 11, 13, 19, 25]
tchai

<font size="6" color='E0218A'><ins>Βήμα 4. Μηχανή αναζήτησης (Search Engine): </ins></font>

Δημιουργούμε παράθυρο tkinter, το οποίο αποτελεί τη μηχανή αναζήτησης.

Ο χρήστης μπορεί να επιλέξει μεταξύ των αλγορίθμων Boolean retrieval (τα αποτελέσματα του οποίου κατατάσσονται με τον αλγόριθμο tf-idf), Vector Space Model και Okapi BM25.

In [39]:
import tkinter as tk
import PIL    #pip install pillow
from tkinter import *
from PIL import ImageTk
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi
import random
root = Tk()
root.title("Barbie search engine")
frame = Frame(root)
frame.pack()

boool= [ "and", "or", "not"]
canvas = Canvas(frame, bg="#ffc5d3", width=1200, height=800)
canvas.pack()

photoimage = ImageTk.PhotoImage(file="barbiepng2.png")

algorithm = StringVar(root)
algorithm.set("Boolean retrieval") # default value
combo = OptionMenu(root, algorithm, "Boolean retrieval", "Vector Space Model", "Okapi BM25")
combo.place(x=750, y=335)

def search(query):
    words = (query).split()
    results=[]
    prevres=[]
    flag=0
    words = [x.lower() for x in words]

    algo= algorithm.get()
    print("Searching with", algo)
    if(algo=="Boolean retrieval"):
        for word in words:
            if (word in dictionary) and (flag==0):
                flag=0
                results.extend(dictionary[word])
                
            elif word in boool:
                boolword=word
                flag=1
            elif (word in dictionary) and (flag!=0):
                prevres=results
                results=[]
                results.extend(dictionary[word])
                flag=0 
                #print("res1", prevres, "res2", results)

                results=boolsearch(prevres, boolword, results)
            else:
                print("Error: words not connected by operators or not in dictionary")
        results_tfidf=tfidf(words, results)
        #print(results_tfidf)
        sorted_results = sorted(results_tfidf.items(), key=lambda x: x[1],reverse=True) #reverse=True makes it descending
        #print("Sorted:", sorted_results)
        results=[x[0] for x in sorted_results]      
    elif(algo=="Vector Space Model"):
        results=VSM(words)
    else:
        results=bm25ok(words)

    #print(results)
    return results

canvas.create_image(600, 150, image=photoimage)  

def printer(results):
    resBox.delete(1.0,END)
    titles=[]
    for i in range(0,len(results)) :
        #i=''.join(i)
       titles.append(data["Title"][results[i]-1])
       resBox.insert(END, data["Title"][results[i]-1])
       resBox.insert(END, "\n")
    

def retrieve_input():
    inputValue=textBox.get("1.0","end-1c")
    res=search(inputValue)
    printer(res)


textBox=Text(root, height=1, width=30, fg="#DA1884")
textBox.pack()
textBox.place(relx=0.5, rely=0.45, anchor='s')
B = Button(root, bg= "#DA1884", fg="white", height=1, width=10, text ="Search", command=lambda: retrieve_input())
B.place(relx=0.5, rely=0.50, anchor='s')
resBox=Text(root, height=10, width=50, fg="#DA1884")
resBox.pack()
resBox.place(relx=0.5, rely=0.75, anchor='s')


def boolsearch(res1, boolword, res2):
    boolres=[]
    if boolword=="and":
        boolres= list(set(res1) & set(res2))
    elif boolword=="or":
        boolres= list(set(res1) | set(res2))
    elif boolword=="not":
        boolres= list(set(res1) - set(res2))
    return(boolres)  

def tfidf(query,files):
    #TF
    tf = {}
    for word in query:
        if word in dictionary:
            tf[word]={}     
            for i in files:   
                tf[word][i]=data["Summary"][i-1].lower().count(word)
    #print(tf)
    #DF
    df = {}
    for word in query:
        frq = 0
        if word in dictionary:
            for doc in dictionary[word]:
                    frq = frq + 1
            df[word] = frq
    #print(df)

    #IDF
    length=len(data["Summary"])
    idf= {} 
    for word in query:
        if word in dictionary:     
            idf[word] = np.log10((length) / df[word])
    #print(idf)

    #TF-IDF
    tf_idf={}
    for word in query:
        if word in dictionary:
            tf_idf[word]={}
            for i in files:
                tf_idf[word][i]=(1+np.log10(tf[word][i])) * idf[word] 

    results={}
    for i in files:
        results[i]=0
        for word in query:
            if word in dictionary:            
                results[i]=results[i]+tf_idf[word][i]
    #print(tf_idf)    
    return results

def bm25ok(query):
    
    tokenized_corpus = [doc.split(" ") for doc in data["Summary"]]
    bm25 = BM25Okapi(tokenized_corpus)
    scores = bm25.get_scores(query)
    #print(scores)
    highest_rank=0
    store=-1
    for i in range(len(scores)):
        if scores[i]>=highest_rank:
            highest_rank=scores[i]
            store=i
    #print("Okapi25 top 1:", store+1)
    results=bm25.get_top_n(query,tokenized_corpus, n=1)
    #print(results)
    return [store+1]
     
  
def VSM(query):
   
    preprocessed_documents = cleaner_words
    tfidf_vectorizer = TfidfVectorizer()


    tfidf_matrix = tfidf_vectorizer.fit_transform(preprocessed_documents)

    query_vector = tfidf_vectorizer.transform(query)

    cosine_similarities=cosine_similarity(query_vector, tfidf_matrix)
    #print(cosine_similarities)

    results=[]
    results = [(i, cosine_similarities[0][i]) for i in range(len(data["Summary"]))]
    
    results.sort(key=lambda x: x[1], reverse=True)

    top_results=[]
    for i in results[0:10]:
        top_results.append(i[0]+1)


    #print(top_results)
    return top_results


frame.mainloop()

Searching with Vector Space Model


<font size="6" color='E0218A'><ins>Βήμα 5. Αξιολόγηση συστήματος: </ins></font>

Λόγω της μορφής των δεδομένων, και του τρόπου συγγραφής των αλγορίθμων (οι οποίοι παίρνουν δεδομένα κατευθείαν από το barbie_movies.csv), η εφαρμογή μετρικών αξιολόγησης στην παραπάνω μηχανή αναζήτησης είναι δύσκολη.

Για αυτό το λόγο, αξιολογείται το έτοιμο σύνολο δεδομένων CISI:

In [44]:
def mean_reciprocal_rank(bool_results, k=10):
   
    bool_results = (np.atleast_1d(r[:k]).nonzero()[0] for r in bool_results)
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in bool_results])

doc_set = {}
doc_id = ""
doc_text = ""
with open('CISI.ALL') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
doc_count = 0
for l in lines:
    if l.startswith(".I"):
        doc_id = int(l.split(" ")[1].strip())-1
    elif l.startswith(".X"):
        doc_set[doc_id] = doc_text.lstrip(" ")
        doc_id = ""
        doc_text = ""
    else:
        doc_text += l.strip()[3:] + " " # The first 3 characters of a line can be ignored.    

            
#Processing QUERIES
with open('CISI.QRY') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
        
qry_set = {}
qry_id = ""
for l in lines:
    if l.startswith(".I"):
        qry_id = int(l.split(" ")[1].strip()) -1
    elif l.startswith(".W"):
        qry_set[qry_id] = l.strip()[3:]
        qry_id = ""

#QRELS
rel_set = {}
with open('CISI.REL') as f:
    for l in f.readlines():
        qry_id = int(l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[0]) -1
        doc_id = int(l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[-1])-1
        if qry_id in rel_set:
            rel_set[qry_id].append(doc_id)
        else:
            rel_set[qry_id] = []
            rel_set[qry_id].append(doc_id)

print('Read %s documents, %s queries and %s mappings from CISI dataset' % (len(doc_set), len(qry_set), len(rel_set)))

number_of_rel_docs = [len(value) for key, value in rel_set.items()]
print('\nAverage %.2f and %d min number of relevant documents by query ' % 
        (np.mean(number_of_rel_docs), np.min(number_of_rel_docs)))

print('\nQueries without relevant documents: ', 
        np.setdiff1d(list(qry_set.keys()),list(rel_set.keys())))
random.seed(42)
idx = random.sample(list(rel_set.keys()),1)[0]

print('\nQuery ID %s ==>' % idx, qry_set[idx])
rel_docs = rel_set[idx]
print('\nDocuments relevants to Query ID %s' % idx, rel_docs)
sample_document_idx = random.sample(rel_docs,1)[0]
print('\nDocument ID %s ==>' % sample_document_idx, doc_set[sample_document_idx])
query = qry_set[idx] #get query text
rel_docs = rel_set[idx] #get relevant documents

# Index all documents using BM25
corpus = list(doc_set.values())
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

# Process query and get scores for each indexed document using BM25
tokenized_query = query.split(" ")
print('\nQuery ==> ', query, '\nRelevant documents IDs: ==> ', rel_docs)
scores = bm25.get_scores(tokenized_query)
print(scores, len(scores), len(doc_set))
most_relevant_documents = np.argsort(-scores)

print("The 20 most relevant documents:", most_relevant_documents[:20]) # printing first 20 most relevant results

 # Mask relevant documents with 0's and 1's according to query <-> document annotation
masked_relevance_results = np.zeros(most_relevant_documents.shape)
masked_relevance_results[rel_docs] = 1
sorted_masked_relevance_results = np.take(masked_relevance_results, most_relevant_documents)

print(sorted_masked_relevance_results[:20]) #printing first 20 results: 1 is relevant 0 isn't


def results_from_query(qry_id, bm25):
   
    query = qry_set[qry_id]
    rel_docs = []
    if qry_id in rel_set:
        rel_docs = rel_set[qry_id]
    tokenized_query = query.split(" ")
    scores = bm25.get_scores(tokenized_query)
    most_relevant_documents = np.argsort(-scores)
    masked_relevance_results = np.zeros(most_relevant_documents.shape)
  
    masked_relevance_results[rel_docs] = 1
    sorted_masked_relevance_results = np.take(masked_relevance_results, most_relevant_documents)
    
    return sorted_masked_relevance_results


results = [results_from_query(qry_id, bm25) for qry_id in list(qry_set.keys())]
print('\n\nMean reciprocal rank total: %.4f' % mean_reciprocal_rank(results))

# Calculate Mean reciprocal rank (top 10)
print("\n\nMean reciprocal rank of sorted results:", mean_reciprocal_rank([sorted_masked_relevance_results]))

Read 1460 documents, 112 queries and 76 mappings from CISI dataset

Average 40.97 and 1 min number of relevant documents by query 

Queries without relevant documents:  [ 35  37  39  46  47  50  52  58  59  62  63  67  69  71  72  73  74  76
  77  79  82  84  85  86  87  88  90  92  93 102 104 105 106 107 109 111]

Query ID 14 ==> How much do information retrieval and dissemination systems, as well as automated libraries, cost? Are they worth it to the researcher and to industry?

Documents relevants to Query ID 14 [17, 26, 35, 48, 55, 58, 66, 73, 82, 125, 157, 163, 166, 191, 213, 221, 222, 249, 280, 291, 294, 298, 306, 330, 335, 337, 347, 364, 365, 366, 367, 371, 380, 445, 457, 464, 465, 481, 489, 490, 494, 496, 506, 519, 527, 590, 593, 622, 628, 638, 689, 719, 722, 723, 726, 727, 730, 778, 821, 833, 838, 847, 848, 864, 871, 896, 1099, 1160, 1247, 1304, 1352, 1357, 1362, 1365, 1367, 1370, 1371, 1373, 1374, 1375, 1376, 1409]

Document ID 48 ==> Adaptive Information Dissemination Sage, 